In [1]:
from bs4 import BeautifulSoup as bs4
import pandas as pd
import requests
import csv

In [3]:
def scrape_all_products(pages=3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    
    all_products = []

    for page in range(1, pages + 1):
        url = f"https://www.chocolate.co.uk/collections/all?page={page}"

        try:
            send_request = requests.get(url, headers=headers)
            send_request.raise_for_status()
            soup = bs4(send_request.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch page {page}: {e}")
            continue

        
        chocolate_cards = soup.find_all('div', class_='product-item__info')
        print(f"Found {len(chocolate_cards)} products on page {page}")

        for card in chocolate_cards:
           
            title_div = card.find('div', class_='product-item-meta')
            title_tag = title_div.find('a') if title_div else None
            title = title_tag.text.strip() if title_tag else "No Title"
            product_url = title_tag['href'] if title_tag else None

          
            price_tag = card.find('div', class_='product-item-meta__price-list-container')
            price_span = price_tag.find('span', class_='price') if price_tag else None
            price = price_span.text.strip() if price_span else "No Price"

           
            if product_url:
                product_page = requests.get(f'https://www.chocolate.co.uk{product_url}', headers=headers)
                product_soup = bs4(product_page.text, 'html.parser')
                ingredients = "No Ingredients"
                
                ingredients_div = product_soup.find('div', class_='metafield-rich_text_field')
                if ingredients_div:
                    for p in ingredients_div.find_all('p'):
                        if 'Ingredients:' in p.text:
                            ingredients = p.text.replace("Ingredients:", "").strip()
                            break
            else:
                ingredients = "No URL"
            full_url=f'https://www.chocolate.co.uk{product_url}'
           
            product = {
                "Title": title,
                "Price": price,
                "Ingredients": ingredients,
                "URL": full_url
            }

            all_products.append(product)

    df = pd.DataFrame(all_products)
    print(f"\nScraped {len(df)} products in total.\n")
    print(df.head())

    return df


data = scrape_all_products(3)

Found 24 products on page 1
Found 24 products on page 2
Found 6 products on page 3

Scraped 54 products in total.

                                     Title                 Price  \
0  2.5kg Bulk 41% Milk Hot Chocolate Drops      Sale price£50.00   
1  2.5kg Bulk 61% Dark Hot Chocolate Drops      Sale price£50.00   
2             41% Milk Hot Chocolate Drops       Sale price£8.75   
3             61% Dark Hot Chocolate Drops       Sale price£8.75   
4                           Almost Perfect  Sale priceFrom £1.00   

                                         Ingredients  \
0  Milk Chocolate (41% Cocoa Minimum, Pure Cocoa ...   
1  Dark Chocolate (61% Cocoa Minimum, Pure Cocoa ...   
2  Milk Chocolate (41% Cocoa Minimum, Pure Cocoa ...   
3  Dark Chocolate (61% Cocoa Minimum, Pure Cocoa ...   
4                                     No Ingredients   

                                                 URL  
0  https://www.chocolate.co.uk/products/2-5kg-bul...  
1  https://www.chocolate.co.u

In [4]:
data.to_csv('New_chocolate_product_dataset.csv', index=False)

In [5]:
data

,Title,Price,Ingredients,URL
0,2.5kg Bulk 41% Milk Hot Chocolate Drops,Sale price£50.00,"Milk Chocolate (41% Cocoa Minimum, Pure Cocoa ...",https://www.chocolate.co.uk/products/2-5kg-bul...
1,2.5kg Bulk 61% Dark Hot Chocolate Drops,Sale price£50.00,"Dark Chocolate (61% Cocoa Minimum, Pure Cocoa ...",https://www.chocolate.co.uk/products/2-5kg-of-...
2,41% Milk Hot Chocolate Drops,Sale price£8.75,"Milk Chocolate (41% Cocoa Minimum, Pure Cocoa ...",https://www.chocolate.co.uk/products/41-colomb...
3,61% Dark Hot Chocolate Drops,Sale price£8.75,"Dark Chocolate (61% Cocoa Minimum, Pure Cocoa ...",https://www.chocolate.co.uk/products/62-dark-h...
4,Almost Perfect,Sale priceFrom £1.00,No Ingredients,https://www.chocolate.co.uk/products/almost-pe...
5,Assorted Chocolate Malt Balls,Sale price£9.00,"White Chocolate (25%) (32% Cocoa Minimum, Pure...",https://www.chocolate.co.uk/products/assorted-...
6,Blonde Caramel,Sale price£5.00,"White Chocolate (35% Cocoa Minimum, Pure Cocoa...",https://www.chocolate.co.uk/products/blonde-ca...
7,Blonde Chocolate Honeycomb - Bag,Sale price£8.50,"White Chocolate (Min 35% Cocoa Min, Pure Cocoa...",https://www.chocolate.co.uk/products/blonde-ch...
8,Blonde Chocolate Malt Balls,Sale price£9.00,"White Chocolate (32% Cocoa Minimum, Pure Cocoa...",https://www.chocolate.co.uk/products/blonde-ch...
9,Blonde Chocolate Truffles,Sale price£19.95,"White Chocolate (32% Cocoa Minimum, Pure Cocoa...",https://www.chocolate.co.uk/products/blonde-ch...
